In [1]:
from preprocessing.prep import prepare_paths, prepare_labels, remover_sublistas_redundantes, split_dataset

2024-04-27 12:06:45.701824: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-27 12:06:46.355247: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from dataset.dataset import __split_data__

In [3]:
import pandas as pd

In [4]:
import os
import json
from tqdm import tqdm

In [5]:

args = pd.Series({
    "root_dir": "/mnt/disks/data/",
    "dataset_path": "/mnt/disks/data/fma/fma_large", 
    "embeddings": "music_style",
    "sequence_size": 1280,
    "train_id": "hierarchical_hiclass_tworoots",
    'sample_size': 1
})



In [56]:
args, tracks_df = prepare_paths(args)

In [58]:
tracks_df.track_genre_top.value_counts()

track_genre_top
Rock                   14172
Experimental           10603
Electronic              9358
Hip-Hop                 3546
Folk                    2801
Pop                     2331
Instrumental            2077
International           1375
Classical               1230
Jazz                     571
Old-Time / Historic      549
Spoken                   421
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: count, dtype: int64

In [61]:
tracks_df = tracks_df[tracks_df['track_genre_top'].isin(['Rock','Electronic'])]

In [62]:
tracks_df

,track_genres_all,track_id,file_path,index,track_genre_top
39284,[12],59906,/mnt/disks/data/fma/fma_large/059/059906.mp3,40162,Rock
54258,"[337, 183, 236, 15]",82335,/mnt/disks/data/fma/fma_large/082/082335.mp3,55652,Electronic
75140,"[296, 468, 15]",115722,/mnt/disks/data/fma/fma_large/115/115722.mp3,77001,Electronic
18607,"[12, 85]",30163,/mnt/disks/data/fma/fma_large/030/030163.mp3,19001,Rock
30958,"[98, 12]",47945,/mnt/disks/data/fma/fma_large/047/047945.mp3,31623,Rock
...,...,...,...,...,...
85336,"[26, 12]",129341,/mnt/disks/data/fma/fma_large/129/129341.mp3,87381,Rock
35908,"[240, 297, 15]",55161,/mnt/disks/data/fma/fma_large/055/055161.mp3,36662,Electronic
3624,[15],7011,/mnt/disks/data/fma/fma_large/007/007011.mp3,3647,Electronic
91676,"[101, 167, 12, 45, 109, 53, 25, 31]",137630,/mnt/disks/data/fma/fma_large/137/137630.mp3,93849,Rock


In [63]:
tracks_df, args = prepare_labels(tracks_df,args)

max depth: 4


/home/bruno/git/fma_prep/preprocessing/prep.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df['full_genre_id'] = estruturas


0it [00:00, ?it/s]

In [64]:
#args.labels

In [65]:
tracks_df

,track_id,full_genre_id
39284,59906,"[[12, , , ]]"
54258,82335,"[[15, 337, , ]]"
75140,115722,"[[15, 296, , ]]"
18607,30163,"[[12, 85, , ]]"
30958,47945,"[[12, 98, , ]]"
...,...,...
85336,129341,"[[12, 26, , ]]"
35908,55161,"[[15, 297, 240, ]]"
3624,7011,"[[15, , , ]]"
91676,137630,"[[12, 31, 101, ]]"


In [66]:
max_depth = tracks_df.full_genre_id.apply(lambda x: max([len(value) for value in x]))
max_depth = int(max_depth.max())

In [67]:
max_depth

4

In [68]:
args.labels

{'label_1': {12: 0, 15: 1},
 'label_1_name': {12: 'Rock', 15: 'Electronic'},
 'label_1_inverse': [12, 15],
 'label_1_count': 2,
 'label_2': {337: 1,
  296: 2,
  85: 3,
  98: 4,
  66: 5,
  440: 6,
  25: 7,
  42: 8,
  297: 9,
  31: 10,
  27: 11,
  236: 12,
  182: 13,
  58: 14,
  495: 15,
  286: 16,
  45: 17,
  181: 18,
  314: 19,
  26: 20,
  468: 21,
  70: 22,
  183: 23,
  185: 24,
  88: 25,
  184: 26,
  359: 27,
  695: 28,
  36: 29},
 'label_2_name': {337: 'Electronic>Drum & Bass',
  296: 'Electronic>Dance',
  85: 'Rock>Garage',
  98: 'Rock>Progressive',
  66: 'Rock>Indie-Rock',
  440: 'Rock>Rock Opera',
  25: 'Rock>Punk',
  42: 'Electronic>Ambient Electronic',
  297: 'Electronic>Chip Music',
  31: 'Rock>Metal',
  27: 'Rock>Lo-Fi',
  236: 'Electronic>IDM',
  182: 'Electronic>House',
  58: 'Rock>Psych-Rock',
  495: 'Electronic>Downtempo',
  286: 'Electronic>Trip-Hop',
  45: 'Rock>Loud-Rock',
  181: 'Electronic>Techno',
  314: 'Rock>Goth',
  26: 'Rock>Post-Rock',
  468: 'Electronic>Dubste

In [69]:
args

root_dir                                                   /mnt/disks/data/
dataset_path                                  /mnt/disks/data/fma/fma_large
embeddings                                                      music_style
sequence_size                                                          1280
train_id                                      hierarchical_hiclass_tworoots
sample_size                                                               1
job_path                  /mnt/disks/data/fma/trains/hierarchical_hiclas...
tfrecord_path             /mnt/disks/data/fma/trains/hierarchical_hiclas...
models_path                                          /mnt/disks/data/models
metadata_path                              /mnt/disks/data/fma/fma_metadata
metadata_train_path       /mnt/disks/data/fma/trains/hierarchical_hiclas...
categories_labels_path    /mnt/disks/data/fma/trains/hierarchical_hiclas...
max_depth                                                                 4
labels      

In [70]:
split_dataset(tracks_df, args)

oversampling_size: 2


100%|████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 1309.00it/s]
2024-04-27 12:40:38.432924: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9786 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5
2024-04-27 12:40:56.897764: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


0 1903 /mnt/disks/data/fma/trains/hierarchical_hiclass_tworoots/tfrecords/val/0000000000.tfrecord
1/1 batchs / 51200 processed
1/1 batchs / 1903 processed
0 4726 /mnt/disks/data/fma/trains/hierarchical_hiclass_tworoots/tfrecords/test/0000000000.tfrecord
1/1 batchs / 51200 processed
1/1 batchs / 4726 processed
0 16900 /mnt/disks/data/fma/trains/hierarchical_hiclass_tworoots/tfrecords/train/0000000000.tfrecord
1/1 batchs / 51200 processed
1/1 batchs / 16900 processed
